In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import random
import pickle
import pandas as pd
import cv2
import os
import ntpath

In [0]:
!git clone https://github.com/abidaks/track

In [0]:
datadir = 'track'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', -1)
data.head()


In [0]:
def get_tail(path):
    head, tail = ntpath.split(path)
    return tail

data['center'] = data['center'].apply(get_tail)
data['left'] = data['left'].apply(get_tail)
data['right'] = data['right'].apply(get_tail)

data.head()

In [0]:
num_bins = 25;
samples_per_bin = 400
hist, bins = np.histogram(data['steering'], num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))

In [0]:
remove_list = []
for j in range(num_bins):
    list_ = []
    for i in range(len(data)):
        if data['steering'][i] >= bins[j] and data['steering'][i] <= bins[j+1]:
            list_.append(i)
    list_ = shuffle(list_)
    list_ = list_[samples_per_bin:]
    remove_list.extend(list_)
    
print("total data:", len(data))
print("removed:",len(remove_list))
data.drop(data.index[remove_list], inplace=True)
print("left data:", len(data))

hist, _ = np.histogram(data['steering'], num_bins)
plt.bar(center, hist, width=0.05)

In [0]:
def load_img_data(datadir, df):
    image_paths = []
    steering = []
    for i in range(len(df)):
        indexed_data = df.iloc[i]
        center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
        image_paths.append(os.path.join(datadir, center.strip()))
        steering.append(float(indexed_data[3]))
    image_paths = np.asarray(image_paths)
    steering = np.asarray(steering)
    return image_paths, steering


In [0]:
image_paths, steering = load_img_data(datadir + '/IMG', data)

x_train, x_valid, y_train, y_valid = train_test_split(image_paths, steering, test_size=0.2, random_state=6)
print("training data:", len(x_train))
print("validation data:", len(x_valid))

In [0]:
fig, axis = plt.subplots(1, 2, figsize=(12, 4))
axis[0].hist(y_train, bins=num_bins, width=0.05, color='blue')
axis[0].set_title('Training set')
axis[1].hist(y_valid, bins=num_bins, width=0.05, color='red')
axis[1].set_title('validation set')

In [0]:
def zoom_img(img):
  zoom = iaa.Affine(scale=(1, 1.3))
  img = zoom.augment_image(img)
  return img

In [0]:
original_img = mpimg.imread(image_paths[100])
z_img = zoom_img(original_img)

fig, axis = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axis[0].imshow(original_img)
axis[0].set_title('original image')
axis[1].imshow(z_img)
axis[1].set_title('zoom image')

In [0]:
def pan_img(img):
  pan = iaa.Affine(translate_percent = {'x': (-0.1, 0.1), 'y': (-0.1, 0.1)})
  img = pan.augment_image(img)
  return img

In [0]:
original_img = mpimg.imread(image_paths[100])
p_img = pan_img(original_img)

fig, axis = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axis[0].imshow(original_img)
axis[0].set_title('original image')
axis[1].imshow(p_img)
axis[1].set_title('pan image')

In [0]:
def random_brightness(img):
  bright = iaa.Multiply((0.2, 1.2))
  img = bright.augment_image(img)
  return img

In [0]:
original_img = mpimg.imread(image_paths[100])
b_img = random_brightness(original_img)

fig, axis = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axis[0].imshow(original_img)
axis[0].set_title('original image')
axis[1].imshow(b_img)
axis[1].set_title('bright image')

In [0]:
def flipping(img, steering_angle):
  img = cv2.flip(img, 1)
  steering_angle = -steering_angle
  return img, steering_angle

In [0]:
original_img = mpimg.imread(image_paths[100])
steering_angle = steering[100]
fl_img, steering_angle = flipping(original_img, steering_angle)

fig, axis = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axis[0].imshow(original_img)
axis[0].set_title('original image')
axis[1].imshow(fl_img)
axis[1].set_title('flipped image')

In [0]:
def augment_image(img, steering_angle):
  img = mpimg.imread(img)
  if np.random.rand() < 0.5:
    img = zoom_img(img)
  if np.random.rand() < 0.5:
    img = pan_img(img)
  if np.random.rand() < 0.5:
    img = random_brightness(img)
  if np.random.rand() < 0.5:
    img, steering_angle = flipping(img, steering_angle)
  
  return img, steering_angle

In [0]:
fig, axis = plt.subplots(10, 2, figsize=(15, 50))
fig.tight_layout()
for i in range(10):
  random_i = random.randint(0, len(image_paths) - 1)
  random_img = image_paths[random_i]
  random_steering = steering[random_i]
  original_img = mpimg.imread(random_img)
  aug_img, random_steering = augment_image(random_img, random_steering)
  
  axis[i][0].imshow(original_img)
  axis[i][0].set_title('original image')
  axis[i][1].imshow(aug_img)
  axis[i][1].set_title('aug image' + str(random_steering))

In [0]:
def preprocess_img(img):
    img = img[60:135, :, :]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = cv2.resize(img, (200,66))
    img = img/255
    return img

In [0]:
original_img = mpimg.imread(image_paths[100])
prep_img = preprocess_img(original_img)

fig, axis = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axis[0].imshow(original_img)
axis[0].set_title('original image')
axis[1].imshow(prep_img)
axis[1].set_title('preprocess image')

In [0]:
def batch_generator(image_paths, steering_angles, batch_size, is_training):
  
  while True:
    batch_img = []
    batch_steering = []
    for i in range(batch_size):
      random_index = random.randint(0, len(image_paths) - 1)
      if is_training:
        im, steering = augment_image(image_paths[random_index], steering_angles[random_index])
      else:
        im = mpimg.imread(image_paths[random_index])
        steering = steering_angles[random_index]
      im = preprocess_img(im)
      batch_img.append(im)
      batch_steering.append(steering)
    yield(np.asarray(batch_img), np.asarray(batch_steering))

In [0]:
def nvidia_model():
    model = Sequential()
    model.add(Convolution2D(24, (5, 5), strides=(2,2), input_shape=(66,200,3), activation="elu"))
    model.add(Convolution2D(36, (5, 5), strides=(2,2), activation="elu"))
    model.add(Convolution2D(48, (5, 5), strides=(2,2), activation="elu"))
    model.add(Convolution2D(64, (3, 3), activation="elu"))
    model.add(Convolution2D(64, (3, 3), activation="elu"))
    #model.add(Dropout(0.5))
    
    model.add(Flatten())
    
    model.add(Dense(100, activation="elu"))
    #model.add(Dropout(0.5))
    model.add(Dense(50, activation="elu"))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation="elu"))
    #model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['accuracy'])
    return model

In [0]:
model = nvidia_model()
print(model.summary())

In [0]:
history = model.fit_generator(batch_generator(x_train, y_train, 100, 1), steps_per_epoch=300, epochs=10, 
                              validation_data=batch_generator(x_valid, y_valid, 100, 0), validation_steps=200,
                              verbose=1, shuffle=1)

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['accuracy', 'val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')

In [0]:
model.save('model.h5')

In [0]:
from google.colab import files
files.download('model.h5')